# Diffusion Model for Open Dialogue

Simplified version

References
- DiffuSeq (cited below)

@inproceedings{gong2022diffuseq,
  author = {Gong, Shansan and Li, Mukai and Feng, Jiangtao and Wu, Zhiyong and Kong, Lingpeng},
  booktitle = {International Conference on Learning Representations, ICLR},
  title = {{DiffuSeq}: Sequence to Sequence Text Generation with Diffusion Models},
  year = 2023
}

@article{gong2023diffuseqv2,
  title={DiffuSeq-v2: Bridging Discrete and Continuous Text Spaces for Accelerated Seq2Seq Diffusion Models},
  author={Gong, Shansan and Li, Mukai and Feng, Jiangtao and Wu, Zhiyong and Kong, Lingpeng},
  journal={arXiv preprint arXiv:2310.05793},
  year={2023}
}


## Dataset

- Use Commonsense Conversation dataset (from Reddit)


in diffuseq text_datasets.py some steps to load the dataset itself

- [ ] prepare datasets for training and validation in the format (stored as jsonl file?)
```
{"src": "", "train": ""}
```

- word embeddings (to be loaded?)
- use a corpus


## Training

Note that, in DiffuSeq, a model file is created to store all training progress, configuration etc. (in bash format poitning to raw files?)

- denoise rate ?
- using updates in v2 diffuseq took it from 2 days -> 11 hr learning time

Load the tokenizer

For simplicity choose BERT rather than custom

In [3]:
# Load tokenizer from bert
# TODO check if this is the best tokenizer for Commonsense Conversation dataset
import torch
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# sep_token_id = tokenizer.sep_token_id
# pad_token_id = tokenizer.pad_token_id


/Users/sallykalumba/dev/diffusion-dialogue/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Diffusion-LM (Li et al., 2022):

 embedding function $EMB(w)$ to map the discrete text $w$ into a continuous space.

load the model embeddings
```
model_weight, tokenizer = load_model_emb(args, tokenizer)
```

Note the implementation details in DiffuSeq (first version) is
"The maximum sequence length is 128, with embedding dimension d = 128, diffusion steps T = 2000
and a square-root noise schedule."

How is it different in v2 or other papers?

In [4]:
# choose hidden_dim

# choose embedding dimension
d=128 
# TODO choose embedding dimension suitable to the task

model = torch.nn.Embedding(tokenizer.vocab_size, 128)

# initialize random embeddings
torch.nn.init.normal_(model.weight)

# torch.save(model.state_dict(), path_save)
# os.sync() FIXME required?

# FIXME need to implement saving and reloading?


Parameter containing:
tensor([[-0.0962, -0.3031,  1.5284,  ..., -1.3276, -0.0819, -1.0058],
        [-0.1696, -0.0784, -0.3533,  ..., -0.4709, -0.3085,  0.2178],
        [-0.6556, -0.8428, -0.9366,  ..., -0.8686,  0.8089, -1.7151],
        ...,
        [-1.5646,  0.9844, -0.1127,  ...,  0.8457,  0.1526,  0.2610],
        [-0.0377,  0.6209, -0.4907,  ...,  0.5751, -1.3167, -1.1007],
        [ 2.3527, -1.1108, -1.1719,  ...,  0.7536, -0.9737,  1.2695]],
       requires_grad=True)

In [5]:
tokenizer.vocab_size


30522

load the text data from file

In [9]:
# read in the data in training data json file 
# TODO do this in a different way 
# FIXME what is the .strip() used for specifically?

import json

data_dir = "./datasets/sample"
path = f'{data_dir}/train.jsonl'

sentence_lst = {'src':[], 'trg': []}
with open(path, 'r') as f_reader:
        for row in f_reader:
            content = json.loads(row)
            sentence_lst['src'].append(content['src'].strip())
            sentence_lst['trg'].append(content['trg'].strip())



# TODO use pandas to load faster? any other package can just load json directly rather than row by row

In [10]:

# Tokenize dataset

# NOTE custom toknizer is of type dict in DiffuSeq

input_id_x = tokenizer(sentence_lst['src'], add_special_tokens=True)['input_ids']
input_id_y = tokenizer(sentence_lst['trg'], add_special_tokens=True)['input_ids']
tokenized_dataset = {'input_id_x': input_id_x, 'input_id_y': input_id_y}

# TODO use batching & multiple processees to make more efficient, use map function?

In [ ]:
# TODO crate a DATALOADER

#  if split != 'test':
#         sampler = DistributedSampler(dataset)
#         data_loader = DataLoader(
#             dataset,
#             batch_size=batch_size,  # 20,
#             # drop_last=True,
#             sampler=sampler,
#             # shuffle=not deterministic,
#             num_workers=4,
#         )
#     else:
#         data_loader = DataLoader(
#             dataset,
#             batch_size=batch_size,  # 20,
#             # drop_last=True,
#             # sampler=sampler,
#             shuffle=not deterministic,
#             num_workers=4,
#         )


In [6]:
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.distributed import DistributedSampler

data_loader = DataLoader(
    tokenized_dataset,
    batch_size=20,  # TODO choose an appropriate batch size ?
    # shuffle=, # TODO choose if you want to shuffle
    # num_workers=4, # TODO use multiple workers throughout script?
)

NameError: name 'tokenized_dataset' is not defined

create the model and diffusion
```
model, diffusion = create_model_and_diffusion(
        **args_to_dict(args, load_defaults_config().keys())
````

create schedule sampler? is it required?

run the training loop according to batch size, etc.